### Managed Care Enrollment by Program and Plan

Data URL: https://data.medicaid.gov/Enrollment/Managed-Care-Enrollment-by-Program-and-Plan/ghyh-r8nn/data

References on state MCO regions:
* IL https://www.illinois.gov/hfs/MedicalProviders/cc/icp/Pages/Regions.aspx
* IL https://www.illinois.gov/hfs/MedicalProviders/cc/icp/Pages/Regions.aspx
* FL https://ahca.myflorida.com/medicaid/statewide_mc/pdf/mma/SMMC_Plans_by_Region.pdf
* KY https://www.researchgate.net/figure/Kentuckys-8-Regions-including-passport-counties-Region-3-and-Kentucky-Health-Select_fig1_261765275

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
df = pd.read_csv(r'2017_Managed_Care_Enrollment_by_Program_and_Plan.csv')

# clean up bad data "Washington9"
df.loc[df['State'] == 'Washington9', 'State'] = 'Washington'

# subset to 3 states
df_sub = df[df['State'].isin(['Florida', 'Illinois', 'Kentucky'])]
grp = df_sub.groupby(['State', 'Geographic Region', 'Plan Name', 'Parent Organization']).agg({'Total Enrollment': 'sum'})
grp = pd.DataFrame(grp).reset_index()

# Enrollment by State, Region, Plan (subset IL, KY, FL)
aggregates across Mcaid "Program" types

In [3]:
# Enrollment by State, Region, Plan (aggregates across Mcaid "Program" types)
grp.sort_values(by=['State', 'Geographic Region', 'Total Enrollment'], ascending=[True, True, False])

,State,Geographic Region,Plan Name,Parent Organization,Total Enrollment
0,Florida,"Collier, Charlotte, and Lee counties",Hope Select Care,Hope Healthcare,322
1,Florida,Miami-Dade County,Florida PACE Center,Miami Jewish Health,683
2,Florida,Palm Beach County,"Morselife Home Care, Inc.",Palm Beach PACE,323
3,Florida,Pinellas County,"Suncoast Neighborly Care, Inc.",Empath Health,287
4,Florida,Region 10,Community Care Plan,South Florida Community Care Network,44766
6,Florida,Region 11,"Simply Healthcare Plans, Inc.","Anthem, Inc.",81284
5,Florida,Region 11,"Coventry Healthcare of FL, Inc.",Aetna,57188
7,Florida,"Regions 1, 2, 3, 5, 6, 7, 8, 9, 10, and 11",Clear Health Alliance,"Anthem, Inc.",9378
8,Florida,"Regions 1, 3,4, 5, 6, 7, 8, 9, 10, and 11","Sunshine State Health Plan, Inc.",Centene,41489
9,Florida,"Regions 1, 4, 6, 7, 8, 9, and 11","Molina Healthcare of Florida, Inc.","Molina Healthcare of Florida, Inc.",346196


In [4]:
# group by State and Plan, sum Enrollment
grp = df.groupby(['State', 'Plan Name']).agg({'Total Enrollment': 'sum'})

# group by State, sum Enrollment
grp2 = df.groupby(['State']).agg({'Total Enrollment': 'sum'})

# get % of State (calculated field)
grp['% of State Enrollees'] = grp / grp2

# Enrollment by state and plan

In [5]:
# enrollment by state and plan
report = grp.reset_index().sort_values(['State', 'Total Enrollment'], ascending=[True, False])

report.head(10)

,State,Plan Name,Total Enrollment,% of State Enrollees
3,Alabama,Patient 1st,621899,0.721539
0,Alabama,Health Homes,224987,0.261034
1,Alabama,Maternity Program,14848,0.017227
2,Alabama,Mercy Life of Alabama,172,0.000200
17,Arizona,United Healthcare Plan,527985,0.326409
13,Arizona,Mercy Care Plan,373819,0.231101
9,Arizona,Health Choice Arizona,254258,0.157186
5,Arizona,Care1st Health Plan,152786,0.094455
19,Arizona,University Family Care,135614,0.083839
11,Arizona,Health Net Access,59822,0.036983


# States where at least 1 plan has > X% of enrollees 

(proxy for lack of options)

In [6]:
thresh = 0.5

_bool = report['% of State Enrollees'] > thresh
report[_bool]

,State,Plan Name,Total Enrollment,% of State Enrollees
3,Alabama,Patient 1st,621899,0.721539
21,Arkansas,Net Transportation,522252,0.526413
175,Delaware,Highmark Health Options of Delaware,117154,0.562643
259,Kentucky,NEMT-Human Services Transportation Delivery,1251301,0.500046
371,Mississippi,Magnolia Health,250836,0.514851
386,Montana,Multiple Primary Care Providers,175012,1.000000
396,Nevada,Medical Transportation Management (MTM Nevada),578683,0.542988
399,New Hampshire,Well Sense,72528,0.542018
505,North Dakota,Multiple Primary Care Providers (PCCM),49399,0.703209
510,Ohio,CareSource,1348067,0.528368


# % of Enrollees in "Non-Competitive" States

In [7]:
non_competitive_states = report[_bool].loc[:, 'State'].drop_duplicates()
non_comp_enroll = df.loc[df['State'].isin(non_competitive_states), 'Total Enrollment'].sum()
total_us_enroll = df['Total Enrollment'].sum()
rate = non_comp_enroll / total_us_enroll

print('enrollees in non-competitive states: {:>20,}'.format(non_comp_enroll))
print('enrollees in overall US Medicaid:    {:>20,}'.format(total_us_enroll))
print('rate:                                {:>19.2f}%'.format(rate*100))

enrollees in non-competitive states:           12,420,940
enrollees in overall US Medicaid:              94,599,851
rate:                                              13.13%


# Enrollment by State

In [8]:
state_grp = df.groupby(['State']).agg({'Total Enrollment': 'sum'})
nat_grp = df['Total Enrollment'].sum()
state_grp['% of national'] = state_grp / nat_grp

# formatting
state_grp['Total Enrollment'] = state_grp['Total Enrollment'].apply(lambda x: '{:,}'.format(x))
state_grp['% of national'] = state_grp['% of national'].apply(lambda x: '{:.2f}%'.format(x*100))

state_grp

,Total Enrollment,% of national
State,,
Alabama,"861,906",0.91%
Arizona,"1,617,558",1.71%
Arkansas,"992,095",1.05%
California,"11,709,927",12.38%
Colorado,"2,500,974",2.64%
Delaware,"208,221",0.22%
District of Columbia,"271,100",0.29%
Florida,"3,280,341",3.47%
Georgia,"1,256,809",1.33%
